In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-jan-2021/train.csv
/kaggle/input/tabular-playground-series-jan-2021/test.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
#L'objectif de cette NoteBook est de tester directement les modéles
#les opérations suivantes sont déja faites précédement dans mon ordinateur

In [3]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')
sub = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv')

In [4]:
train.drop(['id'],inplace=True,axis=1)

In [5]:
test.drop(['id'],inplace=True,axis=1)

In [6]:
#on va essayer de créer des variables plus corrélés a la variable "target"
train['cont3&4'] = train['cont3'] + train['cont4']
train['cont7&8'] = train['cont7'] + train['cont8']
train['cont11&12'] = train['cont11'] + train['cont12']
test['cont3&4'] = test['cont3'] + test['cont4']
test['cont7&8'] = test['cont7'] + test['cont8']
test['cont11&12'] = test['cont11'] + test['cont12']

train['somme1'] = train['cont1'] + train['cont2'] + train['cont3'] + train['cont4'] 
train['somme2'] = train['cont5'] + train['cont6'] + train['cont7'] + train['cont7'] 

test['somme1'] = test['cont1'] + test['cont2'] + test['cont3'] + test['cont4'] 
test['somme2'] = test['cont5'] + test['cont6'] + test['cont7'] + test['cont7'] 



In [10]:
train['v2'] = 0
for i in range(train.shape[0]):
  if train.cont2.iloc[i]  > 0.5 :
    train.v2.iloc[i] = 1

test['v2'] = 0
for i in range(test.shape[0]):
  if( test.cont2.iloc[i]  > 0.5 ):
    test.v2.iloc[i] = 1

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [11]:
train.drop(['v1'],axis=1,inplace=True)

In [12]:
from sklearn.preprocessing import StandardScaler
numerical_cols = ['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8',
       'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14',
       'cont3&4', 'cont7&8', 'cont11&12', 'somme1', 'somme2', 'v2']
for c in numerical_cols:
    prep = StandardScaler()
    train[c] = prep.fit_transform(train[[c]])
    test[c] = prep.transform(test[[c]])

In [13]:
x = train.drop(['target'],axis=1)
y = train['target']
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.15,random_state=42)

In [14]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout

In [15]:
model1 = Sequential()
model1.add(Dense(128,'relu'))
model1.add(Dropout(0.5))
model1.add(Dense(64,'relu'))
model1.add(Dropout(0.3))
model1.add(Dense(32,'relu'))
model1.add(Dropout(0.2))
model1.add(Dense(16,'relu'))
model1.add(Dropout(0.1))
model1.add(Dense(1,'linear'))

model1.compile(optimizer='adam',loss='mse',metrics=[keras.metrics.RootMeanSquaredError()])


In [16]:
model1.fit(
        x_train, y_train,
        validation_data=(x_test, y_test),
        batch_size=10000,
        epochs=1500,
    )

Epoch 1/1500
26/26 [==============================] - 1s 38ms/step - loss: 50.6921 - root_mean_squared_error: 7.1198 - val_loss: 24.0587 - val_root_mean_squared_error: 4.9050
Epoch 2/1500
26/26 [==============================] - 1s 30ms/step - loss: 14.1986 - root_mean_squared_error: 3.7681 - val_loss: 5.3667 - val_root_mean_squared_error: 2.3166
Epoch 3/1500
26/26 [==============================] - 1s 37ms/step - loss: 9.0121 - root_mean_squared_error: 3.0020 - val_loss: 3.4814 - val_root_mean_squared_error: 1.8658
Epoch 4/1500
26/26 [==============================] - 1s 31ms/step - loss: 7.5485 - root_mean_squared_error: 2.7475 - val_loss: 3.1743 - val_root_mean_squared_error: 1.7817
Epoch 5/1500
26/26 [==============================] - 1s 30ms/step - loss: 6.5732 - root_mean_squared_error: 2.5638 - val_loss: 2.8943 - val_root_mean_squared_error: 1.7013
Epoch 6/1500
26/26 [==============================] - 1s 31ms/step - loss: 5.8135 - root_mean_squared_error: 2.4111 - val_loss: 2.46

26/26 [==============================] - 1s 30ms/step - loss: 2.0564 - root_mean_squared_error: 1.4340 - val_loss: 0.5945 - val_root_mean_squared_error: 0.7711
Epoch 49/1500
26/26 [==============================] - 1s 30ms/step - loss: 2.0510 - root_mean_squared_error: 1.4321 - val_loss: 0.5860 - val_root_mean_squared_error: 0.7655
Epoch 50/1500
26/26 [==============================] - 1s 30ms/step - loss: 2.0288 - root_mean_squared_error: 1.4244 - val_loss: 0.5919 - val_root_mean_squared_error: 0.7693
Epoch 51/1500
26/26 [==============================] - 1s 32ms/step - loss: 2.0132 - root_mean_squared_error: 1.4189 - val_loss: 0.5877 - val_root_mean_squared_error: 0.7666
Epoch 52/1500
26/26 [==============================] - 1s 30ms/step - loss: 2.0170 - root_mean_squared_error: 1.4202 - val_loss: 0.5773 - val_root_mean_squared_error: 0.7598
Epoch 53/1500
26/26 [==============================] - 1s 30ms/step - loss: 2.0138 - root_mean_squared_error: 1.4191 - val_loss: 0.5720 - val_ro

26/26 [==============================] - 1s 30ms/step - loss: 1.5974 - root_mean_squared_error: 1.2639 - val_loss: 0.5564 - val_root_mean_squared_error: 0.7459
Epoch 96/1500
26/26 [==============================] - 1s 30ms/step - loss: 1.5900 - root_mean_squared_error: 1.2610 - val_loss: 0.5531 - val_root_mean_squared_error: 0.7437
Epoch 97/1500
26/26 [==============================] - 1s 30ms/step - loss: 1.5815 - root_mean_squared_error: 1.2576 - val_loss: 0.5561 - val_root_mean_squared_error: 0.7457
Epoch 98/1500
26/26 [==============================] - 1s 30ms/step - loss: 1.5731 - root_mean_squared_error: 1.2542 - val_loss: 0.5544 - val_root_mean_squared_error: 0.7446
Epoch 99/1500
26/26 [==============================] - 1s 30ms/step - loss: 1.5621 - root_mean_squared_error: 1.2498 - val_loss: 0.5554 - val_root_mean_squared_error: 0.7452
Epoch 100/1500
26/26 [==============================] - 1s 30ms/step - loss: 1.5470 - root_mean_squared_error: 1.2438 - val_loss: 0.5508 - val_r

Epoch 142/1500
26/26 [==============================] - 1s 31ms/step - loss: 1.3504 - root_mean_squared_error: 1.1621 - val_loss: 0.5376 - val_root_mean_squared_error: 0.7332
Epoch 143/1500
26/26 [==============================] - 1s 30ms/step - loss: 1.3375 - root_mean_squared_error: 1.1565 - val_loss: 0.5337 - val_root_mean_squared_error: 0.7305
Epoch 144/1500
26/26 [==============================] - 1s 31ms/step - loss: 1.3455 - root_mean_squared_error: 1.1600 - val_loss: 0.5352 - val_root_mean_squared_error: 0.7316
Epoch 145/1500
26/26 [==============================] - 1s 31ms/step - loss: 1.3408 - root_mean_squared_error: 1.1579 - val_loss: 0.5324 - val_root_mean_squared_error: 0.7297
Epoch 146/1500
26/26 [==============================] - 1s 30ms/step - loss: 1.3394 - root_mean_squared_error: 1.1573 - val_loss: 0.5321 - val_root_mean_squared_error: 0.7295
Epoch 147/1500
26/26 [==============================] - 1s 31ms/step - loss: 1.3293 - root_mean_squared_error: 1.1530 - val_l

Epoch 189/1500
26/26 [==============================] - 1s 31ms/step - loss: 1.1810 - root_mean_squared_error: 1.0867 - val_loss: 0.5286 - val_root_mean_squared_error: 0.7270
Epoch 190/1500
26/26 [==============================] - 1s 32ms/step - loss: 1.1741 - root_mean_squared_error: 1.0836 - val_loss: 0.5350 - val_root_mean_squared_error: 0.7314
Epoch 191/1500
26/26 [==============================] - 1s 30ms/step - loss: 1.1814 - root_mean_squared_error: 1.0869 - val_loss: 0.5288 - val_root_mean_squared_error: 0.7272
Epoch 192/1500
26/26 [==============================] - 1s 30ms/step - loss: 1.1692 - root_mean_squared_error: 1.0813 - val_loss: 0.5310 - val_root_mean_squared_error: 0.7287
Epoch 193/1500
26/26 [==============================] - 1s 30ms/step - loss: 1.1598 - root_mean_squared_error: 1.0769 - val_loss: 0.5255 - val_root_mean_squared_error: 0.7249
Epoch 194/1500
26/26 [==============================] - 1s 32ms/step - loss: 1.1588 - root_mean_squared_error: 1.0765 - val_l

...

26/26 [==============================] - 1s 31ms/step - loss: 0.5049 - root_mean_squared_error: 0.7105 - val_loss: 0.5018 - val_root_mean_squared_error: 0.7084
Epoch 1311/1500
26/26 [==============================] - 1s 30ms/step - loss: 0.5043 - root_mean_squared_error: 0.7102 - val_loss: 0.5014 - val_root_mean_squared_error: 0.7081
Epoch 1312/1500
26/26 [==============================] - 1s 31ms/step - loss: 0.5048 - root_mean_squared_error: 0.7105 - val_loss: 0.5009 - val_root_mean_squared_error: 0.7078
Epoch 1313/1500
26/26 [==============================] - 1s 30ms/step - loss: 0.5051 - root_mean_squared_error: 0.7107 - val_loss: 0.5015 - val_root_mean_squared_error: 0.7082
Epoch 1314/1500
26/26 [==============================] - 1s 30ms/step - loss: 0.5049 - root_mean_squared_error: 0.7105 - val_loss: 0.5015 - val_root_mean_squared_error: 0.7082
Epoch 1315/1500
26/26 [==============================] - 1s 30ms/step - loss: 0.5049 - root_mean_squared_error: 0.7105 - val_loss: 0.501

26/26 [==============================] - 1s 31ms/step - loss: 0.5046 - root_mean_squared_error: 0.7104 - val_loss: 0.5012 - val_root_mean_squared_error: 0.7079
Epoch 1357/1500
26/26 [==============================] - 1s 30ms/step - loss: 0.5048 - root_mean_squared_error: 0.7105 - val_loss: 0.5017 - val_root_mean_squared_error: 0.7083
Epoch 1358/1500
26/26 [==============================] - 1s 30ms/step - loss: 0.5051 - root_mean_squared_error: 0.7107 - val_loss: 0.5015 - val_root_mean_squared_error: 0.7082
Epoch 1359/1500
26/26 [==============================] - 1s 30ms/step - loss: 0.5048 - root_mean_squared_error: 0.7105 - val_loss: 0.5014 - val_root_mean_squared_error: 0.7081
Epoch 1360/1500
26/26 [==============================] - 1s 31ms/step - loss: 0.5048 - root_mean_squared_error: 0.7105 - val_loss: 0.5015 - val_root_mean_squared_error: 0.7082
Epoch 1361/1500
26/26 [==============================] - 1s 32ms/step - loss: 0.5053 - root_mean_squared_error: 0.7108 - val_loss: 0.501

26/26 [==============================] - 1s 31ms/step - loss: 0.5047 - root_mean_squared_error: 0.7104 - val_loss: 0.5016 - val_root_mean_squared_error: 0.7082
Epoch 1403/1500
26/26 [==============================] - 1s 31ms/step - loss: 0.5050 - root_mean_squared_error: 0.7106 - val_loss: 0.5013 - val_root_mean_squared_error: 0.7080
Epoch 1404/1500
26/26 [==============================] - 1s 32ms/step - loss: 0.5047 - root_mean_squared_error: 0.7104 - val_loss: 0.5012 - val_root_mean_squared_error: 0.7080
Epoch 1405/1500
26/26 [==============================] - 1s 30ms/step - loss: 0.5048 - root_mean_squared_error: 0.7105 - val_loss: 0.5012 - val_root_mean_squared_error: 0.7080
Epoch 1406/1500
26/26 [==============================] - 1s 30ms/step - loss: 0.5052 - root_mean_squared_error: 0.7108 - val_loss: 0.5013 - val_root_mean_squared_error: 0.7080
Epoch 1407/1500
26/26 [==============================] - 1s 30ms/step - loss: 0.5051 - root_mean_squared_error: 0.7107 - val_loss: 0.501

26/26 [==============================] - 1s 30ms/step - loss: 0.5053 - root_mean_squared_error: 0.7109 - val_loss: 0.5009 - val_root_mean_squared_error: 0.7078
Epoch 1449/1500
26/26 [==============================] - 1s 30ms/step - loss: 0.5050 - root_mean_squared_error: 0.7106 - val_loss: 0.5011 - val_root_mean_squared_error: 0.7079
Epoch 1450/1500
26/26 [==============================] - 1s 30ms/step - loss: 0.5046 - root_mean_squared_error: 0.7104 - val_loss: 0.5007 - val_root_mean_squared_error: 0.7076
Epoch 1451/1500
26/26 [==============================] - 1s 32ms/step - loss: 0.5049 - root_mean_squared_error: 0.7105 - val_loss: 0.5011 - val_root_mean_squared_error: 0.7079
Epoch 1452/1500
26/26 [==============================] - 1s 30ms/step - loss: 0.5045 - root_mean_squared_error: 0.7103 - val_loss: 0.5012 - val_root_mean_squared_error: 0.7080
Epoch 1453/1500
26/26 [==============================] - 1s 31ms/step - loss: 0.5048 - root_mean_squared_error: 0.7105 - val_loss: 0.501

26/26 [==============================] - 1s 30ms/step - loss: 0.5046 - root_mean_squared_error: 0.7104 - val_loss: 0.5006 - val_root_mean_squared_error: 0.7075
Epoch 1495/1500
26/26 [==============================] - 1s 30ms/step - loss: 0.5055 - root_mean_squared_error: 0.7110 - val_loss: 0.5010 - val_root_mean_squared_error: 0.7078
Epoch 1496/1500
26/26 [==============================] - 1s 31ms/step - loss: 0.5047 - root_mean_squared_error: 0.7105 - val_loss: 0.5010 - val_root_mean_squared_error: 0.7078
Epoch 1497/1500
26/26 [==============================] - 1s 30ms/step - loss: 0.5054 - root_mean_squared_error: 0.7109 - val_loss: 0.5012 - val_root_mean_squared_error: 0.7080
Epoch 1498/1500
26/26 [==============================] - 1s 30ms/step - loss: 0.5048 - root_mean_squared_error: 0.7105 - val_loss: 0.5009 - val_root_mean_squared_error: 0.7078
Epoch 1499/1500
26/26 [==============================] - 1s 30ms/step - loss: 0.5045 - root_mean_squared_error: 0.7103 - val_loss: 0.501

In [17]:
y_predd = model1.predict(test) 

In [18]:
#submission
sub['target']= y_predd
sub.to_csv('submission3.csv', index=False)

In [19]:
import os 
os.chdir(r'../working')
from IPython.display import FileLink
FileLink(r'submission3.csv')

/kaggle/working/submission3.csv